# Gradient Boosted Trees for graph data

course: pv056

name: Marek Kadlčík

učo: 485294


In [13]:
from copy import copy
from typing import Dict

import numpy as np
import pandas as pd
import rdkit
import torch
import torch_geometric as pyg

from catboost import CatBoostClassifier
from rdkit.Chem import MACCSkeys
from rdkit import RDLogger

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    precision_recall_curve,
    auc,
    roc_auc_score,
    f1_score,
)


## Loading data

Data is loaded again from torch_geometric, to ensure consistent train-test split with the neural method.

In [14]:
def transform_lipo(sample: pyg.data.Data) -> pyg.data.Data:
    sample = copy(sample)
    sample.y = (sample.y > 3.5).float().squeeze()
    return sample

def transform_hiv(sample: pyg.data.Data) -> pyg.data.Data:
    sample = copy(sample)
    sample.y = sample.y.squeeze()
    return sample

def transform_bppp(sample: pyg.data.Data) -> pyg.data.Data:
    sample = copy(sample)
    sample.y = sample.y.squeeze()
    return sample

def transform_tox21(sample: pyg.data.Data) -> pyg.data.Data:
    sample = copy(sample)
    sample.y = sample.y.squeeze()[2]
    return sample

def transform_clintox(sample: pyg.data.Data) -> pyg.data.Data:
    sample = copy(sample)
    sample.y = sample.y.squeeze()[0]
    return sample


def is_y_not_na(sample: pyg.data.Data) -> bool:
    return not torch.isnan(sample.y)

transforms = {
    "lipo": transform_lipo,
    "hiv": transform_hiv,
    "bbbp": transform_bppp,
    "clintox": transform_clintox,
    "tox21": transform_tox21,
}

datasets = {}

for name, transform in transforms.items():
    print("preparing:", name, end=" ")
    data = pyg.datasets.MoleculeNet("./data/", name)
    data = map(transform, data)
    data = filter(is_y_not_na, data)
    data = pd.Series(data)
    datasets[name] = data

    for i in range(100):
        sample = data[i]
        assert sample.y.shape == tuple()
        assert sample.y.dtype == torch.float32
        assert any([
            torch.isclose(sample.y, torch.tensor(0.0)),
            torch.isclose(sample.y, torch.tensor(1.0))
        ])
    
    print(len(data))

del data

preparing: lipo 4200
preparing: hiv 41127
preparing: bbbp 2039
preparing: clintox 1478
preparing: tox21 6549


## Preprocessing

I will use the smiles string from the dataset. Smiles is a string lossless representation of molecular structure and we can reconstruct a molecule from it. Then I use rdkit to generate MACCS Keys fingerprint. Molecular fingerprinting is a traditional way of computing feature vectors from molecules. MACCS Key contains 166 binary variables.

In [15]:
def smiles2fingerprint(smiles: str) -> np.ndarray:
    mol = rdkit.Chem.MolFromSmiles(smiles)
    bits = MACCSkeys.GenMACCSKeys(mol)
    return np.frombuffer(bytes(bits.ToBitString(), 'utf-8'), 'u1') - ord('0')

In [16]:
def prepare_dataset(data):
    train_split = np.random.default_rng(seed=0).uniform(0, 1, size=len(data))

    # rdkit gives a few warnings when computing the fingerprints
    # on some molecules, but there is not much one can do about it
    RDLogger.DisableLog('rdApp.*')
    df = pd.DataFrame({
        "smiles": [mol.smiles for mol in data],
        "x": [smiles2fingerprint(mol.smiles) for mol in data],
        "y": [mol.y.numpy().ravel().item() for mol in data]
    }).set_index("smiles")
    RDLogger.EnableLog('rdApp.*')

    df_train = df.iloc[train_split < 0.8]
    df_valid = df.iloc[(train_split < 0.9) & (train_split >= 0.8)]
    df_tests = df.iloc[train_split > 0.9]

    return df_train, df_valid, df_tests

## Metrics

Again, the same set of metrics as in neural method. The interesting for us are area under ROC curve and area under precision-recall curve.

In [17]:
def compute_metrics(model, df, threshold) -> Dict[str, float]:
    pred_probs = model.predict_proba(np.stack(df.x))[:,1]
    pred = pred_probs > threshold
    true = df.y

    precisions, recalls, _ = precision_recall_curve(true, pred_probs)
    return {
        "accuracy": accuracy_score(true, pred),
        "recall": recall_score(true, pred),
        "precision": precision_score(true, pred),
        "f1": f1_score(true, pred),
        "roc_auc": roc_auc_score(true, pred_probs),
        "pr_auc": auc(recalls, precisions),
    }

## Training

I chose CatBoost as a nonneural method. It is a popular implementation of gradient boosted trees, has GPU acceleration and has well-thought-out choice of default parameters.

In [20]:
from IPython.display import Markdown, display

models = {}
results = {}

print()
for dataset_name, data in datasets.items():
    
    df_train, df_valid, df_tests = prepare_dataset(data)

    display(Markdown(f"### {dataset_name}"))
    print(flush=True, end="")
    print("training.")

    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_train.y), y=df_train.y)
    print("class weights:", class_weights)

    model = CatBoostClassifier(
        task_type="GPU",
        loss_function="Logloss",
        class_weights=class_weights,
        random_seed=42,
    )

    model.fit(
        np.stack(df_train.x),
        np.stack(df_train.y.to_numpy()),
        eval_set=(df_valid.x, df_valid.y),
        use_best_model=True,
        verbose=False
    )

    models[dataset_name] = model

    threshold = 0.5

    results[dataset_name] = pd.DataFrame({
        "train": compute_metrics(model, df_train, threshold),
        "valid": compute_metrics(model, df_valid, threshold),
        "tests": compute_metrics(model, df_tests, threshold),
    })

    display(results[dataset_name].round(3))
    

### lipo

training.
class weights: [0.56835871 4.15717822]


,train,valid,tests
accuracy,0.870,0.792,0.786
recall,0.973,0.638,0.587
precision,0.481,0.339,0.293
f1,0.644,0.443,0.391
roc_auc,0.975,0.826,0.784
pr_auc,0.845,0.381,0.449


### hiv

training.
class weights: [ 0.51825277 14.19655172]


...

training.
class weights: [7.18518519 0.53739612]


,train,valid,tests
accuracy,0.812,0.822,0.814
recall,0.801,0.827,0.813
precision,0.997,0.985,0.991
f1,0.888,0.899,0.893
roc_auc,0.958,0.814,0.944
pr_auc,0.997,0.989,0.997


### tox21

training.
class weights: [0.56713013 4.22411003]


,train,valid,tests
accuracy,0.868,0.858,0.861
recall,0.930,0.810,0.789
precision,0.472,0.430,0.438
f1,0.626,0.561,0.563
roc_auc,0.959,0.911,0.881
pr_auc,0.796,0.552,0.599
